In [2]:
import pandas as pd
import numpy as np
import math

import tensorflow as tf
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

import pytz
import os
import sys
import pickle
from datetime import datetime

In [3]:
# Define network parameters
def initialize_random_weights(mean, std, shape = ()):
    return np.random.normal(loc=mean, scale=std, size=shape)
        
# Function to generate dataset with multiplication
def generate_dataset_with_zeros(size, n_max=10):
    # Generate two columns of random numbers between 0 and 9
    column_1 = np.random.randint(0, n_max, size)
    column_2 = np.random.randint(0, n_max, size)

    # Create a DataFrame with the two columns
    dataset = pd.DataFrame({
        'Column_1': column_1,
        'Column_2': column_2
    })

    # Create the third column by multiplying the first two
    dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']

    return dataset

def generate_dataset_without_zeros(size, n_max=10):
    # Generate two columns of random numbers between 1 and 9
    column_1 = np.random.randint(1, n_max, size)
    column_2 = np.random.randint(1, n_max, size)

    # Create a DataFrame with the two columns
    dataset = pd.DataFrame({
        'Column_1': column_1,
        'Column_2': column_2
    })

    # Create the third column by multiplying the first two
    dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']

    return dataset

def generate_test_dataset(n_max=10):
    # Create the columns
    column_1 = list(range(n_max)) * n_max  # Numbers from 0 to 9 repeated 10 times
    column_2 = [i for i in range(n_max) for _ in range(n_max)]  # Numbers from 0 to 9 repeated sequentially 10 times

    # Create a DataFrame with the two columns
    dataset = pd.DataFrame({
        'Column_1': column_1,
        'Column_2': column_2,
    })

    # Create the third column by multiplying the first two
    dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']

    return dataset

def decimal_to_binary(n, bits):
    if 0 <= n < 2**bits:
        # Convert the number to a binary string and then to an array of integers (0 and 1)
        return np.array(list(format(n, f'0{bits}b'))).astype(np.int8)
    else:
        raise ValueError("Number out of range")

# Function to convert binary number to decimal
def binary_to_decimal(binary_vector, bits):
    # Ensure the vector has the correct number of elements
    if len(binary_vector) != bits:
        raise ValueError(f"The vector must have exactly {bits} elements.")

    # Calculate the decimal number
    decimal = 0
    for i in range(bits):
        decimal += binary_vector[i] * (2 ** (bits - 1 - i))

    return decimal

def transform_to_tridimensional_matrix(dataset, bits_init=4, bits_end=7):
    rows, cols = dataset.shape
    if cols != 3:
        raise ValueError("The dataset must have exactly 3 columns.")

    # Initialize the three matrices
    matrix_column_1 = np.zeros((rows, bits_init), dtype=np.int8)
    matrix_column_2 = np.zeros((rows, bits_init), dtype=np.int8)
    matrix_column_3 = np.zeros((rows, bits_end), dtype=np.int8)

    # Fill the matrices with the binary representation of each column
    for i in range(rows):
        matrix_column_1[i] = decimal_to_binary(dataset.iloc[i, 0], bits_init)
        matrix_column_2[i] = decimal_to_binary(dataset.iloc[i, 1], bits_init)
        matrix_column_3[i] = decimal_to_binary(dataset.iloc[i, 2], bits_end)

    return matrix_column_1, matrix_column_2, matrix_column_3
    
def prepare_dataset(level, size=1, couples_included=[]): 
    if level == -3:
        column_1 = []
        column_2 = []
        pairs = couples_included
        while len(column_1) < size:
            choice = pairs[np.random.choice(len(pairs))]
            column_1.append(choice[0])
            column_2.append(choice[1])
        dataset = pd.DataFrame({'Column_1': column_1,'Column_2': column_2,})
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    elif level == -2:
        dataset = generate_dataset_with_zeros(size)
        return dataset
        
    elif level == -1:
        dataset = generate_dataset_without_zeros(size)
        return dataset

    elif level == 0:
        couples_not_included = [(3, 3), (5, 5), (6, 6), (7, 7), (9, 9), (3, 6), (3, 7), (6, 3), (7, 3), (5, 7), (7, 5), (6, 7), (7, 6)]
        dataset = pd.DataFrame()
        while len(dataset) < size:
            column_1 = np.random.randint(1, 10, size)
            column_2 = np.random.randint(1, 10, size)
            temp_dataset = pd.DataFrame({'Column_1': column_1, 'Column_2': column_2})
            temp_dataset = temp_dataset[~temp_dataset[['Column_1', 'Column_2']].apply(tuple, axis=1).isin(couples_not_included)]
            dataset = pd.concat([dataset, temp_dataset])
        dataset = dataset.iloc[:size].reset_index(drop=True)
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    elif level == 1:
        column_1 = []
        column_2 = []
        pairs = [(5, 5), (9, 9)]
        while len(column_1) < size:
            choice = pairs[np.random.choice(len(pairs))]
            column_1.append(choice[0])
            column_2.append(choice[1])
        dataset = pd.DataFrame({'Column_1': column_1,'Column_2': column_2,})
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    elif level == 2:
        column_1 = []
        column_2 = []
        pairs = [(3, 3), (6, 6), (3, 6), (6, 3), (5, 7), (7, 5)]
        while len(column_1) < size:
            choice = pairs[np.random.choice(len(pairs))]
            column_1.append(choice[0])
            column_2.append(choice[1])
        dataset = pd.DataFrame({'Column_1': column_1,'Column_2': column_2,})
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    elif level == 3:
        column_1 = []
        column_2 = []
        pairs = [(3, 7), (6, 7), (7, 3), (7, 6)]
        while len(column_1) < size:
            choice = pairs[np.random.choice(len(pairs))]
            column_1.append(choice[0])
            column_2.append(choice[1])
        dataset = pd.DataFrame({'Column_1': column_1,'Column_2': column_2,})
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    elif level == 4:
        column_1 = [7] * size
        column_2 = [7] * size
        dataset = pd.DataFrame({'Column_1': column_1,'Column_2': column_2,})
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    elif level == 5:
        column_1 = []
        column_2 = []
        pairs = [(3, 3), (5, 5), (6, 6), (7, 7), (9, 9), (3, 6), (3, 7), (6, 3), (7, 3), (5, 7), (7, 5), (6, 7), (7, 6)]
        while len(column_1) < size:
            choice = pairs[np.random.choice(len(pairs))]
            column_1.append(choice[0])
            column_2.append(choice[1])
        dataset = pd.DataFrame({'Column_1': column_1,'Column_2': column_2,})
        dataset['Column_3'] = dataset['Column_1'] * dataset['Column_2']
        return dataset

    else:
        print('Bad index for the training stage.')
        return None


In [56]:
def train_neural_network(model, level, epochs=10, steps_per_epoch=100, batch_size=100):
    decimal_dataset_train = prepare_dataset(level, size=steps_per_epoch*batch_size)
    x_train_1, x_train_2, y_train = transform_to_tridimensional_matrix(decimal_dataset_train)
    x_train = np.concatenate([x_train_1, x_train_2], axis=1)
    
    decimal_dataset_test = generate_test_dataset(n_max=11)
    x_val_1, x_val_2, y_val = transform_to_tridimensional_matrix(decimal_dataset_test)
    x_val = np.concatenate([x_val_1, x_val_2], axis=1)
    
    x_train = np.expand_dims(x_train, axis=1)
    x_val = np.expand_dims(x_val, axis=1)
    x_train = x_train.astype(np.float32)
    y_train = y_train.astype(np.float32)
    x_val = x_val.astype(np.float32)
    y_val = y_val.astype(np.float32)
    validation_data = (x_val, y_val)
    
    history = model.fit(
        x_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=validation_data,
        verbose=1
    )
    
    return history

In [57]:
def test_neural_network(model, visualize_errors = 0, real_test = 0):
    if real_test == 0:
        carryover_list = [(3, 3), (5, 5), (6, 6), (7, 7), (9, 9), (3, 6), (3, 7), (6, 3), (7, 3), (5, 7), (7, 5), (6, 7), (7, 6)]
        decimal_dataset = generate_test_dataset()
    elif real_test == 1:
        carryover_list = [(3, 3), (5, 5), (6, 6), (7, 7), (9, 9), (3, 6), (3, 7), (6, 3), (7, 3), (5, 7), (7, 5), (6, 7), (7, 6),
                             (5, 10), (10, 5), (7, 10), (10, 7), (10, 10), (10, 11), (11, 10), (3, 11), (11, 3), (5, 11), (11, 5),
                             (6, 11), (11, 6), (7, 11), (11, 7), (9, 11), (11, 9), (11, 11)]
        decimal_dataset = generate_test_dataset(n_max=12)

    x_test_1, x_test_2, y_train = transform_to_tridimensional_matrix(decimal_dataset)
    x_test = np.concatenate([x_test_1, x_test_2], axis=1)
    x_test = np.expand_dims(x_test, axis=1)
    predictions = trainable_model.predict(x_test)
    rounded_predictions = np.round(predictions).astype(int)

    correct_predictions_count = 0
    carryover_mistakes = 0  
    test_size = x_test.shape[0]

    for i in range(test_size):
        if np.all(rounded_predictions[i] == y_train[i]):  # Check if the prediction matches the expected output
            correct_predictions_count += 1
        else:
            if (decimal_dataset.iloc[i,0], decimal_dataset.iloc[i,1]) in carryover_list:
                carryover_mistakes += 1
            if visualize_errors == 1:
                print(f'Mistake: {decimal_dataset.iloc[i,0]} times {decimal_dataset.iloc[i,1]}.')

    return test_size, correct_predictions_count, carryover_mistakes

In [84]:
class RoundingLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return K.round(inputs)  # Round to 0 and 1

def generate_model():
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    initializer1 = tf.keras.initializers.GlorotNormal()
    model = tf.keras.Sequential()
    inputs = tf.keras.Input(shape=(None, 8))

    # Construir las capas a partir de 'inputs'
    x = tf.keras.layers.LSTM(128, return_sequences=True, kernel_initializer=initializer1)(inputs)
    x = tf.keras.layers.Dropout(0.05)(x)
    x = tf.keras.layers.LSTM(512)(x)
    x = tf.keras.layers.Dropout(0.05)(x)
    x = tf.keras.layers.LSTM(256)(inputs)
    x = tf.keras.layers.Dropout(0.05)(x)
    x = tf.keras.layers.LSTM(128)(inputs)
    x = tf.keras.layers.Dropout(0.05)(x)
    outputs = tf.keras.layers.Dense(7, activation='sigmoid')(x)

    # Crear el modelo completo
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    model.summary()
    return model


In [85]:
trainable_model = generate_model()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 8)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │          70,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 71,047 (277.53 KB)

 Trainable params: 71,047 (277.53 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
train_neural_network(trainable_model, level=-2, epochs=150, steps_per_epoch=300, batch_size=100)

Epoch 1/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1576 - loss: 0.2065 - val_accuracy: 0.1818 - val_loss: 0.1443
Epoch 2/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step - accuracy: 0.2068 - loss: 0.1168 - val_accuracy: 0.2975 - val_loss: 0.1051
Epoch 3/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - accuracy: 0.2805 - loss: 0.0813 - val_accuracy: 0.3388 - val_loss: 0.0857
Epoch 4/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step - accuracy: 0.3120 - loss: 0.0597 - val_accuracy: 0.3884 - val_loss: 0.0715
Epoch 5/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.3360 - loss: 0.0453 - val_accuracy: 0.3554 - val_loss: 0.0605
Epoch 6/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step - accuracy: 0.3423 - loss: 0.0355 - val_accuracy: 0.3636 - val_loss: 0.0535
Epoch 7/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step - accuracy: 0.3422 - loss: 0.0292 - val_accuracy: 0.3554 - val_loss: 0.0490
Epoch 8/150
300/300 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.3505 - loss: 0.0

In [88]:
visualize_errors = 0

for real_test in range(0, 2):
    test_size, correct_predictions_count, carryover_mistakes = test_neural_network(trainable_model, visualize_errors=visualize_errors, real_test=real_test)
    print(f"Out of {test_size}, {correct_predictions_count} were predicted correctly in the current model, with {carryover_mistakes} mistakes in examples with carryover.")



4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Out of 100, 94 were predicted correctly in the current model, with 6 mistakes in examples with carryover.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
Out of 144, 107 were predicted correctly in the current model, with 24 mistakes in examples with carryover.
